In [2]:
import pandas as pd
import numpy as np
from fancyimpute import IterativeImputer

In [146]:
df0 = pd.read_csv("./csv_folder/mimic_processed.csv", index_col = 0,engine = 'python')
df0.shape

(13346, 70)

In [88]:
df0.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime',
       'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'first_careunit',
       'weight_first', 'height_first', 'icu_mortality', 'vent_duration_days',
       'nettotal', 'hypertension', 'congestive_heart_failure', 'diabetes',
       'hadm_id.1', 'afib', 'ckd', 'cld', 'copd', 'cereb', 'ihd', 'cancer',
       'aki_48hr', 'sepsis', 'vaspor', 'vaspin', 'apsiii', 'apsiii_prob',
       'sofa', 'first_24_vasso_hours', 'first_24_vasso_counts',
       'first_48_vasso_hours', 'first_48_vasso_counts', 'first_72_vasso_hours',
       'first_72_vasso_counts', 'exclude_vaso', 'exclude_age',
       'ethnicity_classification', 'exclude_height_first',
       'exclude_weight_first', 'bmi', 'exclude_los_mortality', 'heartrate',
       'resprate', 'exclude_heartrate', 'exclude_resprat

In [97]:
df1 = df0[(df0['exclude_age']==False) ]
df1.shape

(12917, 70)

In [93]:
df0[(df0['exclude_age']==True)].shape

(429, 70)

In [95]:
df0['hospital_expire_flag'].unique()

array([0, 1])

In [105]:
df2= df1[(df1['ibp_mean_24h'].notnull()) & (df1['ibp_systolic_24h'].notnull()) & (df1['ibp_diastolic_24h'].notnull())]
df2.shape

(12726, 70)

In [103]:
df1[(df1['ibp_mean_24h'].isnull()) | (df1['ibp_systolic_24h'].isnull()) | (df1['ibp_diastolic_24h'].isnull())].shape


(191, 70)

In [106]:
df2.ethnicity.unique()

array(['WHITE', 'HISPANIC OR LATINO', 'UNKNOWN/NOT SPECIFIED',
       'UNABLE TO OBTAIN', 'BLACK/AFRICAN AMERICAN', 'ASIAN', 'OTHER',
       'MULTI RACE ETHNICITY', 'BLACK/HAITIAN',
       'HISPANIC/LATINO - GUATEMALAN', 'ASIAN - CHINESE',
       'PATIENT DECLINED TO ANSWER', 'MIDDLE EASTERN',
       'ASIAN - CAMBODIAN', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER',
       'HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC/LATINO - DOMINICAN',
       'ASIAN - ASIAN INDIAN', 'BLACK/CAPE VERDEAN', 'BLACK/AFRICAN',
       'WHITE - BRAZILIAN', 'WHITE - RUSSIAN', 'ASIAN - VIETNAMESE',
       'HISPANIC/LATINO - CUBAN', 'ASIAN - OTHER', 'ASIAN - FILIPINO',
       'HISPANIC/LATINO - COLOMBIAN', 'PORTUGUESE',
       'HISPANIC/LATINO - MEXICAN', 'ASIAN - KOREAN',
       'AMERICAN INDIAN/ALASKA NATIVE', 'WHITE - OTHER EUROPEAN',
       'WHITE - EASTERN EUROPEAN', 'SOUTH AMERICAN', 'ASIAN - JAPANESE',
       'HISPANIC/LATINO - SALVADORAN', 'ASIAN - THAI'], dtype=object)

In [107]:
def ethnicity_clustering(ethnicity_original):
    if ethnicity_original.startswith('ASIAN'):
        return 'ASIAN'
    elif ethnicity_original.startswith('WHITE'):
        return 'WHITE'
    elif ethnicity_original.startswith('HISPANIC'):
        return 'HISPANIC'
    elif ethnicity_original.startswith('BLACK'):
        return 'BLACK'
    else:
        return 'OTHERS'

In [108]:
df2['ethnicity_new'] = df2['ethnicity'].apply(ethnicity_clustering)

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
df2.ethnicity_new.value_counts()

WHITE       8858
OTHERS      2431
BLACK        768
HISPANIC     376
ASIAN        293
Name: ethnicity_new, dtype: int64

In [110]:
df2.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime',
       'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'first_careunit',
       'weight_first', 'height_first', 'icu_mortality', 'vent_duration_days',
       'nettotal', 'hypertension', 'congestive_heart_failure', 'diabetes',
       'hadm_id.1', 'afib', 'ckd', 'cld', 'copd', 'cereb', 'ihd', 'cancer',
       'aki_48hr', 'sepsis', 'vaspor', 'vaspin', 'apsiii', 'apsiii_prob',
       'sofa', 'first_24_vasso_hours', 'first_24_vasso_counts',
       'first_48_vasso_hours', 'first_48_vasso_counts', 'first_72_vasso_hours',
       'first_72_vasso_counts', 'exclude_vaso', 'exclude_age',
       'ethnicity_classification', 'exclude_height_first',
       'exclude_weight_first', 'bmi', 'exclude_los_mortality', 'heartrate',
       'resprate', 'exclude_heartrate', 'exclude_resprat

In [111]:
df2.loc[df2['first_24_vasso_hours'].isna(),'first_24_vasso_hours']=0
df2.loc[df2['first_48_vasso_hours'].isna(),'first_48_vasso_hours']=0
df2.loc[df2['first_72_vasso_hours'].isna(),'first_72_vasso_hours']=0

/home/ephlius/.virtualenvs/3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [112]:
df3 = df2[['icustay_id','gender','age','ethnicity_new','ibp_mean_24h','ibp_systolic_24h','height_first','weight_first']]
df3 = pd.get_dummies(df3, columns=['gender'])
df3 = pd.get_dummies(df3, columns=['ethnicity_new'])

In [113]:
MICE_imputer = IterativeImputer()
height_MICE = df3.copy(deep=True)
height_MICE.iloc[:, :] = MICE_imputer.fit_transform(height_MICE)

In [114]:
df_impute = height_MICE.filter(['icustay_id','height_first','weight_first'],axis=1)
df_impute.columns = ['icustay_id','height_impute','weight_impute']

In [115]:
df4 = pd.merge(df2, df_impute, how='left', on=['icustay_id'])

In [116]:
df4.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime',
       'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'first_careunit',
       'weight_first', 'height_first', 'icu_mortality', 'vent_duration_days',
       'nettotal', 'hypertension', 'congestive_heart_failure', 'diabetes',
       'hadm_id.1', 'afib', 'ckd', 'cld', 'copd', 'cereb', 'ihd', 'cancer',
       'aki_48hr', 'sepsis', 'vaspor', 'vaspin', 'apsiii', 'apsiii_prob',
       'sofa', 'first_24_vasso_hours', 'first_24_vasso_counts',
       'first_48_vasso_hours', 'first_48_vasso_counts', 'first_72_vasso_hours',
       'first_72_vasso_counts', 'exclude_vaso', 'exclude_age',
       'ethnicity_classification', 'exclude_height_first',
       'exclude_weight_first', 'bmi', 'exclude_los_mortality', 'heartrate',
       'resprate', 'exclude_heartrate', 'exclude_resprat

In [117]:
df4.shape

(12726, 73)

In [118]:
df4.height_impute.describe()

count    12726.000000
mean       169.612821
std          9.671956
min        121.920000
25%        162.246187
50%        170.180000
75%        177.023220
max        210.820000
Name: height_impute, dtype: float64

In [119]:
height_min = df4['height_impute'].quantile(.25) - 1.5 * (df4['height_impute'].quantile(.75) - df4['height_impute'].quantile(.25)) # 139.8
height_max = df4['height_impute'].quantile(.75) + 1.5 * (df4['height_impute'].quantile(.75) - df4['height_impute'].quantile(.25)) # 200.6

In [120]:
df4['exclude_height'] = False
df4.loc[(df4['height_impute']>height_max )|(df4['height_impute']<height_min ),'exclude_height'] = True

In [29]:
df4[df4['exclude_height']==False].height_impute.describe()

count    12680.000000
mean       169.682517
std          9.441590
min        141.990000
25%        162.344726
50%        170.180000
75%        177.025705
max        198.120000
Name: height_impute, dtype: float64

In [122]:
df4[df4['exclude_height']==True].shape

(46, 74)

In [123]:
weight_min = df4['weight_impute'].quantile(.25) - 1.5 * (df4['weight_impute'].quantile(.75) - df4['weight_impute'].quantile(.25)) # 139.8
weight_max = df4['weight_impute'].quantile(.75) + 1.5 * (df4['weight_impute'].quantile(.75) - df4['weight_impute'].quantile(.25)) # 200.6

In [124]:
df4['exclude_weight'] = False
df4.loc[(df4['weight_impute']>weight_max )|(df4['weight_impute']<weight_min ),'exclude_weight'] = True

In [125]:
df4[df4['exclude_weight']==False].weight_impute.describe()

count    12335.000000
mean        84.684154
std         18.059303
min         36.100000
25%         72.300000
50%         84.000000
75%         95.800000
max        133.900000
Name: weight_impute, dtype: float64

In [126]:
df4[df4['exclude_weight']==True].shape

(391, 75)

In [127]:
df5 = df4[(df4['exclude_height']==False) & (df4['exclude_weight']==False ) ]
df5.shape

(12291, 75)

In [128]:
df4[(df4['exclude_height']==True) | (df4['exclude_weight']==True ) ].shape


(435, 75)

In [129]:
df5.columns

Index(['subject_id', 'hadm_id', 'icustay_id', 'gender', 'admittime',
       'dischtime', 'los_hospital', 'age', 'ethnicity', 'admission_type',
       'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime',
       'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay', 'first_careunit',
       'weight_first', 'height_first', 'icu_mortality', 'vent_duration_days',
       'nettotal', 'hypertension', 'congestive_heart_failure', 'diabetes',
       'hadm_id.1', 'afib', 'ckd', 'cld', 'copd', 'cereb', 'ihd', 'cancer',
       'aki_48hr', 'sepsis', 'vaspor', 'vaspin', 'apsiii', 'apsiii_prob',
       'sofa', 'first_24_vasso_hours', 'first_24_vasso_counts',
       'first_48_vasso_hours', 'first_48_vasso_counts', 'first_72_vasso_hours',
       'first_72_vasso_counts', 'exclude_vaso', 'exclude_age',
       'ethnicity_classification', 'exclude_height_first',
       'exclude_weight_first', 'bmi', 'exclude_los_mortality', 'heartrate',
       'resprate', 'exclude_heartrate', 'exclude_resprat

In [130]:
df6 = df5.filter(['icustay_id','age', 'gender', 'ethnicity_new', 'first_careunit',
                       'height_impute','weight_impute',
                       'icu_mortality', 'los_icu','hospital_expire_flag', 'los_hospital', 
                       'sofa',
                       
                       'first_24_vasso_hours','first_48_vasso_hours', 'first_72_vasso_hours',
                       'hypertension', 'congestive_heart_failure', 'diabetes','afib', 'ckd', 
                       'cld', 'copd', 'cereb', 'ihd', 'cancer','sepsis','vent_duration_days',
                       
                       'ibp_systolic_24h', 'ibp_systolic_48h', 'ibp_systolic_72h',
                       'ibp_diastolic_24h', 'ibp_diastolic_48h', 'ibp_diastolic_72h',
                       'ibp_mean_24h', 'ibp_mean_48h', 'ibp_mean_72h'
                       
                       
                      ],axis=1)

In [131]:
df6['bmi'] = df6['weight_impute'] / (df6['height_impute']/100 * df6['height_impute']/100)

In [132]:
df6.bmi.describe()

count    12291.000000
mean        29.423715
std          5.631681
min         12.620470
25%         25.686882
50%         29.516950
75%         31.930576
max         56.673716
Name: bmi, dtype: float64

In [133]:
df_7day =pd.read_csv("./csv_folder/mimic_new_aki_7day.csv",engine = 'python')
df_7day.head(2)

,icustay_id,charttime_creat,creat,aki_stage_creat,charttime_uo,uo_rt_6hr,uo_rt_12hr,uo_rt_24hr,aki_stage_uo,aki_stage_7day,aki_7day
0,200018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,200020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [134]:
df_final = pd.merge(df6, df_7day[['icustay_id','aki_7day']], on='icustay_id',  how='left')
df_final.loc[df_final['aki_7day'].isna(),'aki_7day']=0
df_final.head(2)

,icustay_id,age,gender,ethnicity_new,first_careunit,height_impute,weight_impute,icu_mortality,los_icu,hospital_expire_flag,...,ibp_systolic_48h,ibp_systolic_72h,ibp_diastolic_24h,ibp_diastolic_48h,ibp_diastolic_72h,ibp_mean_24h,ibp_mean_48h,ibp_mean_72h,bmi,aki_7day
0,271147,54.5208,F,WHITE,SICU,162.408138,83.5,0,1.5940,0,...,99.0,99.0,47.0,50.0,50.0,66.5,68.0,68.0,31.657084,1
1,214619,21.5041,M,HISPANIC,TSICU,173.783414,76.8,0,11.5029,0,...,147.0,148.0,67.0,72.0,72.0,84.0,91.0,92.0,25.429895,0


In [135]:
sum(df_final['aki_7day']==1)/(df_final.shape[0])

0.7992840289642829

In [136]:
df_final.columns

Index(['icustay_id', 'age', 'gender', 'ethnicity_new', 'first_careunit',
       'height_impute', 'weight_impute', 'icu_mortality', 'los_icu',
       'hospital_expire_flag', 'los_hospital', 'sofa', 'first_24_vasso_hours',
       'first_48_vasso_hours', 'first_72_vasso_hours', 'hypertension',
       'congestive_heart_failure', 'diabetes', 'afib', 'ckd', 'cld', 'copd',
       'cereb', 'ihd', 'cancer', 'sepsis', 'vent_duration_days',
       'ibp_systolic_24h', 'ibp_systolic_48h', 'ibp_systolic_72h',
       'ibp_diastolic_24h', 'ibp_diastolic_48h', 'ibp_diastolic_72h',
       'ibp_mean_24h', 'ibp_mean_48h', 'ibp_mean_72h', 'bmi', 'aki_7day'],
      dtype='object')

In [137]:
df_final.shape

(12291, 38)

In [138]:
df_final.icu_mortality.unique()

array([0, 1])

# survivor 7 day

In [139]:
df_final['mortality_7d'] = 2
df_final.loc[df_final['los_icu']>7,'mortality_7d'] = 0
df_final.loc[df_final['los_icu']<=7,'mortality_7d'] = df_final['icu_mortality']


In [140]:
df_final['mortality_7d'].value_counts()

0    11445
1      846
Name: mortality_7d, dtype: int64

In [141]:
df_final.mortality_7d.unique()

array([0, 1])

In [142]:
df_final.isna().any()

icustay_id                  False
age                         False
gender                      False
ethnicity_new               False
first_careunit              False
height_impute               False
weight_impute               False
icu_mortality               False
los_icu                     False
hospital_expire_flag        False
los_hospital                False
sofa                        False
first_24_vasso_hours        False
first_48_vasso_hours        False
first_72_vasso_hours        False
hypertension                False
congestive_heart_failure    False
diabetes                    False
afib                        False
ckd                         False
cld                         False
copd                        False
cereb                       False
ihd                         False
cancer                      False
sepsis                      False
vent_duration_days          False
ibp_systolic_24h            False
ibp_systolic_48h            False
ibp_systolic_7

In [143]:
df_final.shape

(12291, 39)

In [144]:
df_final.to_csv("./csv_folder/mimic_clean_19Aug.csv")

In [145]:
df_final.columns

Index(['icustay_id', 'age', 'gender', 'ethnicity_new', 'first_careunit',
       'height_impute', 'weight_impute', 'icu_mortality', 'los_icu',
       'hospital_expire_flag', 'los_hospital', 'sofa', 'first_24_vasso_hours',
       'first_48_vasso_hours', 'first_72_vasso_hours', 'hypertension',
       'congestive_heart_failure', 'diabetes', 'afib', 'ckd', 'cld', 'copd',
       'cereb', 'ihd', 'cancer', 'sepsis', 'vent_duration_days',
       'ibp_systolic_24h', 'ibp_systolic_48h', 'ibp_systolic_72h',
       'ibp_diastolic_24h', 'ibp_diastolic_48h', 'ibp_diastolic_72h',
       'ibp_mean_24h', 'ibp_mean_48h', 'ibp_mean_72h', 'bmi', 'aki_7day',
       'mortality_7d'],
      dtype='object')